In [ ]:
import numpy as np
import pandas as pd
import json
import os
import glob
import random
import gc
import keras
import mcfly
from keras import backend as K
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import auc, roc_curve, precision_recall_curve, brier_score_loss
from sklearn.calibration import calibration_curve
from datetime import datetime 
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import math
import scipy.stats as st
from scipy import stats
import csv

In [ ]:
#path to file with indexes of files split into training, val and test
split_path = "400_dumped/Final_Data/split/train_val_test.json"

#paths to the labels and the data
labels_path = "400_dumped/Final_Data/labels/labels.npy"
samples_path = "400_dumped/Final_Data/samples/"

#choose model to laod and train
model_name = "Model6.json"
trained_path = "400_dumped/Models_Final_Data/trained/" + model_name
type_path = "400_dumped/Models_Final_Data/type/type_"

#path to save plots
model_name_no_extension = model_name.split(".", 1)[0] + "/"
plot_path = "400_dumped/TestPlots/" +  model_name_no_extension

#path to original json data, to check filter types
path_negative = "AnonymisedECGs_json/negative"
path_positive = "AnonymisedECGs_json/positive"

#set to true to save plots
save_plots = False

#set the seed 
random.seed(0) #generation of train, val, test sets
np.random.seed(0) #mcfly models
tf.random.set_seed(0) #keras training

# Dictionary with sample id and label

In [ ]:
labels_array = np.load(labels_path)
labels = dict()

for row in labels_array:
    labels[row[0]] = int(row[1])

    
del labels_array
gc.collect()

# Train, val, test split

In [ ]:
#open original train, val, test split to calculate original weights
with open(split_path, "r") as fp:
    train_val_test_dict = json.load(fp)

In [ ]:
len(train_val_test_dict["test"])

# Test set selection

In [ ]:
test = train_val_test_dict["test"].copy()

# Check filter distribution 

In [ ]:
pos_filters = pd.DataFrame()
neg_filters = pd.DataFrame()

for elem in test: 
    
    if elem[0] == str(1):
            directory = path_negative + "/"+ elem + ".json"
            
    if elem[0] == str(2):
        directory = path_positive + "/" + elem + ".json"
    
    f = open(directory)
    data = json.load(f)
        
    ecg = data["RestingECG"]
    waveform = pd.DataFrame(ecg["Waveform"])
    waveform_rhythm = pd.DataFrame(waveform[waveform["WaveformType"]=="Rhythm"])
    
    label = ""
    if "positive" in directory:
        label = "positive"
    elif "negative" in directory:
        label = "negative"
    
    temp = pd.DataFrame(
    {
        "id": elem, 
        "high_pass": waveform_rhythm["HighPassFilter"],
        "low_pass": waveform_rhythm["LowPassFilter"],
        "ac": waveform_rhythm["ACFilter"],
        "label": label
    })
    
    if label == "positive":
        pos_filters = pd.concat([pos_filters, temp])
    elif label == "negative":
        neg_filters = pd.concat([neg_filters, temp])
    

In [ ]:
def analyse_filter_dist(df): 
    filter_combo = df.groupby(["high_pass", "low_pass", "ac", "label"]).size().reset_index(name="Count")
    filter_combo["percentage_by_class"] = 100 * filter_combo["Count"] / filter_combo.groupby("label")["Count"].transform("sum")
    filter_combo["combination"] = list(zip(filter_combo.high_pass, filter_combo.low_pass, filter_combo.ac))
    filter_combo = filter_combo.sort_values(by=["label", "percentage_by_class"], ascending=False)
    
    return filter_combo

In [ ]:
p_filter_combo = analyse_filter_dist(pos_filters)
n_filter_combo = analyse_filter_dist(neg_filters)

In [ ]:
p_filter_combo

In [ ]:
n_filter_combo

In [ ]:
number_negatives = np.sum(n_filter_combo["Count"])
number_positives = np.sum(p_filter_combo["Count"])

In [ ]:
#class imbalance 
class_imb = number_negatives / number_positives

#fraction of positives
pos_fraction = number_positives / (number_positives + number_negatives)
print(class_imb, pos_fraction)

In [ ]:
n_filter_combo = n_filter_combo.set_index("combination")
n_filter_combo = n_filter_combo.reindex(index = p_filter_combo["combination"])
n_filter_combo = n_filter_combo.reset_index()

In [ ]:
p_filter_combo

In [ ]:
n_filter_combo

In [ ]:
n_filter_combo_head = n_filter_combo.head(5)
p_filter_combo_head = p_filter_combo.head(5)

p_filter_combo_head = p_filter_combo_head.set_index("combination")
p_filter_combo_head = p_filter_combo_head.reindex(index = n_filter_combo_head["combination"])
p_filter_combo_head = p_filter_combo_head.reset_index()

In [ ]:
ind = np.arange(p_filter_combo_head.shape[0])
width = 0.35

fig, ax = plt.subplots(figsize=(20, 12.5))
rects_neg = ax.bar(ind - width/2, n_filter_combo_head["percentage_by_class"], width, label = "Negative")
rects_pos = ax.bar(ind + width/2, p_filter_combo_head["percentage_by_class"], width, label = "Positive")
ax.set_ylabel("Percentage of samples")
ax.set_title("Top 5 percentage of samples per filter combination per class")
ax.set_xticks(ind)
y_labels = list(n_filter_combo_head["combination"])
ax.set_xticklabels(y_labels)
ax.legend()

In [ ]:
diff1 = pd.merge(n_filter_combo[["combination", "percentage_by_class"]],
                p_filter_combo[["combination", "percentage_by_class"]],
                how = "left",
                left_on = ["combination"],
                right_on = ["combination"],
                suffixes = ["_neg", "_pos"])
diff1["difference"] = diff1["percentage_by_class_neg"]- diff1["percentage_by_class_pos"]
diff1

# Functions

In [ ]:
#function to create validation and test sets and store in memory
def set_generation(val_or_test, train_val_test_dict, labels, dim = (2500, 8)):
    n_samples = len(train_val_test_dict[val_or_test])

    #Initialise
    X = np.empty((n_samples, dim[0], dim[1]))
    y = np.empty((n_samples), dtype = int)

    #Generate data
    for i, ID in enumerate(train_val_test_dict[val_or_test]):
        #store sample
        X[i,] = np.load(samples_path + ID +".npy")

        #store class
        y[i] = labels[ID]
    
    return X, y

In [ ]:
def performance_metrics(y_true, y_pred, y_proba, metrics):
    conf_mat = confusion_matrix(y_true, y_pred)
    print("Confusion matrix: ")
    print(conf_mat)
    tn,fp,fn,tp = conf_mat.ravel()
    print("tn: ", tn," fp: ", fp," fn: ", fn," tp: ", tp)
    
    print("")
    matthews = ((tp*tn) - (fp*fn)) / math.sqrt(((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    print("Matthews Correlation Coefficient: ", matthews)
    
    print("")
    print(classification_report(y_true, y_pred))
    
    print("")           
    precision_bis = tp/(tp+fp) #positive predictive value
    recall_bis = tp/(tp+fn)
    f1 = 2*precision_bis*recall_bis/(precision_bis+recall_bis)
    specificity = tn/(tn+fp) #true negative rate
    fnr = fn/(fn+tp)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    print("precision/positive predictive value: ", precision_bis)
    print("recall/sensitivity: ", recall_bis)
    print("specificity/true negative rate: ", specificity)
    print("False negative rate: ", fnr)
    print("accuracy: ", accuracy)    
    print("f1 score: ", f1) 

      
    print("")
    brier = brier_score_loss(y_true, y_proba)
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    auc_coef = auc(fpr, tpr)
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    auprc = auc(recall, precision)
    print("brier score: ", brier )
    print("auc: ", auc_coef)
    print("auprc: ", auprc)
    
    metrics.append([tn, fp, fn, tp, matthews, precision_bis, recall_bis, specificity, fnr, accuracy, f1, auc_coef, auprc, brier])
    
    return 


#predicts on cv test set and gets performance stats
def predictions(model, X_test, y_test, metrics):
    pred_probas = model.predict(X_test)

    #BrS appears as 1, hence transformed to [0,1] => the second column returns 1 if BrS, 0 otherwise
    BrS = y_test[:,1]
    
    #get probabilities and predictions
    BrS_probas = pred_probas[:,1]
    BrS_predictions = pred_probas.argmax(axis = -1)
    BrS_predictions
    
    #get performance metrics
    performance_metrics(BrS, BrS_predictions, BrS_probas, metrics)
    
    return BrS, pred_probas, BrS_probas, BrS_predictions

# Predictions

In [ ]:
#load trained model 
model = keras.models.load_model(trained_path)

#generate test set and save to memory
X_test, y_test = set_generation("test", train_val_test_dict, labels, (2500, 8))
y_test = keras.utils.to_categorical(y_test, 2)

#predict and get performance metrics
metrics = []
BrS, pred_probas, BrS_probas, BrS_predictions = predictions(model, X_test, y_test, metrics)

# Plots

In [ ]:
with open(type_path + model_name, "r") as f:
    model_type = json.load(f)    
    print(model_type)

In [ ]:
#ROC curve
fpr, tpr, thresholds = roc_curve(BrS, BrS_probas)
auc_coef = round(auc(fpr, tpr),3)
f, ax = plt.subplots(figsize=(6,6))
ax.plot(fpr, tpr, marker=".", label = model_type["type"] + " - AUC: " + str(auc_coef))
ax.plot([0,1], [0,1], transform = ax.transAxes, linestyle="--", label="Random Classifier")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
#ax.set_title("ROC")
ax.legend()
if save_plots:
    plt.savefig(plot_path + "ROC.png")

In [ ]:
#Precision Recall curve
precision, recall, thresholds = precision_recall_curve(BrS, BrS_probas)
auprc = round(auc(recall, precision),3)
f, ax = plt.subplots(figsize=(6,6))
ax.plot(recall, precision, marker=".", label = model_type["type"] + " - AUPRC: " + str(auprc))
ax.set_xlabel("Recall (Positive label: Brugada)")
ax.set_ylabel("Precision (Positive label: Brugada)")
#ax.set_title("AUPRC")
ax.set_ylim([0.0, 1.05])
ax.legend()

if save_plots:
    plt.savefig(plot_path + "PrecisionRecallCurve.png")

In [ ]:
#Calibration
# bin data and normalise counts
def counts_to_percentages(probabilities):
    bin0_01 = 0
    bin01_02=0
    bin02_03=0
    bin03_04=0
    bin04_05=0
    bin05_06=0
    bin06_07=0
    bin07_08=0
    bin08_09=0
    bin09_1=0 
    
    for val in probabilities:
    
        if val <0.1:
            bin0_01 = bin0_01 + 1
    
        elif val >= 0.1 and val <0.2:
            bin01_02= bin01_02 +1 
    
        elif val >= 0.2 and val <0.3:
            bin02_03= bin02_03 +1 
    
        elif val >= 0.3 and val <0.4:
                bin03_04= bin03_04 +1
    
        elif val >= 0.4 and val <0.5:
                bin04_05= bin04_05 +1 
    
        elif val >= 0.5 and val <0.6:
                bin05_06= bin05_06 +1 
    
        elif val >= 0.6 and val <0.7:
                    bin06_07= bin06_07 +1 
    
        elif val >= 0.7 and val <0.8:
                    bin07_08= bin07_08 +1 
    
        elif val >= 0.8 and val <0.9:
                    bin08_09= bin08_09 +1 
    
        elif val >= 0.9:
                    bin09_1= bin09_1 +1 
                
    counts = [bin0_01, bin01_02, bin02_03, bin03_04, bin04_05,
             bin05_06, bin06_07, bin07_08, bin08_09, bin09_1]    
    
    percentages = counts/np.sum(counts) *100
    
    return percentages
    
    
#plot calibration plot and histogram together
def calibration_together (BrS, BrS_probas, per_patient = False):        
    print("plot curves and save in one png file")
    #save three plots in one png file
    fig_index = 1
      
    #save three plots in one png file
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(7, 12))   
    
    # plot calibration 
    y, x = calibration_curve(BrS, BrS_probas, n_bins=10)

    ax1.plot(x, y, 'C0',marker='o', linewidth=1, label= model_type["type"], color = "darkturquoise") 
    ax1.set(xlabel= 'Predicted score', ylabel= 'True probability in each bin')
    
    line = mlines.Line2D([0, 1], [0, 1], color='black', linestyle='--', linewidth=0.9, label= "Perfectly calibrated")
    transform = ax1.transAxes
    line.set_transform(transform)
    ax1.add_line(line)     
    ax1.legend(loc="upper left")  
  
    #HISTOGRAMS    
    x = np.arange(0,1,0.1)
    y = counts_to_percentages(BrS_probas)   #if instead of % want values in [0,1], do: y = counts_to_percentages(proba)/100 
    ax2.hist(x, range = [0,1], bins=10, weights = y, label= model_type["type"],
                 histtype="step", lw=3.5, color = "darkturquoise")
    
    ax2.set_xlabel("Mean predicted score")
    ax2.set_ylabel("Percentage of counts")
    ax2.legend(loc="upper center", ncol=5)
    ax2.set_ylim([0,101]) #if instead of % want probabilities, change to [0,1]     

    #plt.tight_layout()
    if per_patient: 
        plt.savefig(plot_path + "Calibration_PP.png")
        
    elif save_plots:
        plt.savefig(plot_path + "Calibration.png")
        
    plt.show()
    
    return

calibration_together(BrS, BrS_probas)

In [ ]:
#Discrimination
def distribution(BrS, BrS_probas, per_patient = False):
    #probabilities distributions graphs
    true_1 = pd.DataFrame(BrS_probas, columns=['Predicted probabilities'])
    true_1['labels'] = BrS.tolist()
    true_0 = true_1.copy(deep = True) 
    indexNames = true_1[true_1['labels'] == 0].index
    true_1.drop(indexNames , inplace=True)
    indexNames = true_0[ true_0['labels'] == 1 ].index
    true_0.drop(indexNames , inplace=True)
    true_1.drop(columns=['labels'], inplace = True)
    true_0.drop(columns=['labels'], inplace = True)
    
    sns.distplot(true_1['Predicted probabilities'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3,"color": "r"}, label = 'Class 1')
    sns.distplot(true_0['Predicted probabilities'], hist = False, kde = True,
                     kde_kws = {'shade': True, 'linewidth': 3, "color": "g"}, label = 'Class 0')
    plt.ylabel('Density')
    plt.xlabel('Predicted score')
    plt.legend(labels=["BrP","No BrP"])
    
    if per_patient: 
        plt.savefig(plot_path + "Discrimination_PP.png")
        
    elif save_plots:
        plt.savefig(plot_path + "Discrimination.png")  
        
        
    plt.show()
    plt.clf()    
    return

distribution(BrS, BrS_probas)

# LIME

In [ ]:
from lime import explanation
from lime import lime_base
from lime_timeseries import LimeTimeSeriesExplainer

def custom_predict(trained_model):
    #puts sample in right format for keras prediction
    def func(sample):
        prediction = trained_model.predict(np.transpose(sample, axes=[0,2,1]))
        return prediction
    return func

def explain_sample_lime(explainer,series_ecg, num_features_ecg, n_samples, num_slices_ecg, replacement_method, predict_function = custom_predict(trained_model=model)):
    
    exp = explainer.explain_instance(series_ecg, predict_function, num_features=num_features_ecg, num_samples=n_samples, 
                                 num_slices=num_slices_ecg, replacement_method = "total_mean")
    
    fig = exp.as_pyplot_figure() 
    
    return exp, fig

def plot_lime(series_ecg, num_slices_ecg, X_val, y_val, BrS_predictions, idx_ecg, exp):

    values_per_slice_ecg = math.ceil(series_ecg.shape[1]/ num_slices_ecg)
    no_pattern = X_val[np.where(y_val[:,1]==0)]
    pattern = X_val[np.where(y_val[:,1]==1)]
    lead_to_index = {"I": 0, "II": 1, "V1": 2, "V2":3,
                    "V3": 4, "V4": 5, "V5": 6, "V6":7}
    leads = ["I", "II", "V1","V2","V3","V4","V5","V6"]

    labels = ["no Brugada Pattern", "Brugada Pattern"]
    true_label = labels[int(y_val[idx_ecg,1])]
    predicted_label = labels[BrS_predictions[idx_ecg]]

    fig, axes = plt.subplots(nrows = 4, ncols = 2, figsize = (30,30), sharex = True, sharey=True)

    for i, ax in enumerate(axes.flatten()):
        ax.plot(series_ecg[i], 'b', label='Explained instance')
        ax.plot(no_pattern[:,:,i].mean(axis=0), color='red',label='Mean of class no Brugada Pattern')
        ax.plot(pattern[:,:,i].mean(axis=0), color='green',label='Mean of class Brugada Pattern')
        ax.set_title("Lead "+ leads[i], fontsize = 25)

        for j in range(num_features_ecg):
            feature, weight = exp.as_list()[j]        
            feature_name_index = lead_to_index[feature.split(" ", 2)[2]] #split at second space in feature name to take lead name as key

            if feature_name_index == i:
                start = int(feature.split(" ", 1)[0]) * values_per_slice_ecg #int(feature.split(" ", 1)[0]): only keep int from feature name, eg feature name (23 - II), split at " " (space) and keep first part (23) and take int(23)
                end = start + values_per_slice_ecg
                color = 'red' if weight < 0 else 'green' 
                ax.axvspan(start , end, color=color, alpha=abs(weight*10))

    ax.legend(loc='lower left')
    title = "LIME explanation of single sample. True label: " + true_label + " . Predicted label: " + predicted_label + "."
    fig.suptitle(title, fontsize=50)
    
    if true_label == "no Brugada Pattern" and predicted_label == "no Brugada Pattern":
        saved_title = "True_negative.png"
    elif true_label == "no Brugada Pattern" and predicted_label == "Brugada Pattern":
        saved_title = "False_positive.png"
    elif true_label == "Brugada Pattern" and predicted_label == "no Brugada Pattern":
        saved_title = "False_negative.png"
    elif true_label == "Brugada Pattern" and predicted_label == "Brugada Pattern":
        saved_title = "True_positive.png"
        
    plt.rc("axes", labelsize=25)
    plt.rc("legend", fontsize=20)
    
    #fig.savefig(plot_path + saved_title)
    plt.show()
            
    return fig

#interpretation: real label is no BrP but model predicts as Brugada (false positive). Green bands correspond to
# evidence that it's a positive sample, red bands correspond to evidence that it's a negative sample

In [ ]:
num_features_ecg = 100 #number of lime weights
num_slices_ecg = 50 #number of segments of a lead
n_samples = 50 #number of perturbated samples at a single time point
replacement_method = "total_mean" #possible replacement mathods: "noise" (fill in noise for perturbation), "mean" (fill in mean of segment), "total_mean" (fill in mean of lead)
explainer = LimeTimeSeriesExplainer(class_names = ["No BrP", "BrP"], signal_names= ["I", "II", "V1", "V2", "V3", "V4", "V5", "V6"])

In [ ]:
#tp
idx_ecg = 0 #0th sample
series_ecg = X_test[idx_ecg].T
exp, weights_fig = explain_sample_lime(explainer,series_ecg, num_features_ecg, n_samples, num_slices_ecg, replacement_method)

In [ ]:
plot_lime(series_ecg, num_slices_ecg, X_test, y_test, BrS_predictions, idx_ecg, exp)

In [ ]:
num_features_ecg = 10 #number of lime weights
num_slices_ecg = 50 #number of segments of a lead
n_samples = 50 #number of perturbated samples at a single time point
replacement_method = "total_mean" #possible replacement mathods: "noise" (fill in noise for perturbation), "mean" (fill in mean of segment), "total_mean" (fill in mean of lead)
explainer = LimeTimeSeriesExplainer(class_names = ["No BrP", "BrP"], signal_names= ["I", "II", "V1", "V2", "V3", "V4", "V5", "V6"])

In [ ]:
#tp
idx_ecg = 0 #0th sample
series_ecg = X_test[idx_ecg].T
exp, weights_fig = explain_sample_lime(explainer,series_ecg, num_features_ecg, n_samples, num_slices_ecg, replacement_method)

In [ ]:
plot_lime(series_ecg, num_slices_ecg, X_test, y_test, BrS_predictions, idx_ecg, exp)

In [ ]:
#tp
idx_ecg = 1 #nth sample
series_ecg = X_test[idx_ecg].T
exp, weights_fig = explain_sample_lime(explainer,series_ecg, num_features_ecg, n_samples, num_slices_ecg, replacement_method)

In [ ]:
plot_lime(series_ecg, num_slices_ecg, X_test, y_test, BrS_predictions, idx_ecg, exp)

In [ ]:
#fn
idx_ecg = 56 #nth sample
series_ecg = X_test[idx_ecg].T
exp, weights_fig = explain_sample_lime(explainer,series_ecg, num_features_ecg, n_samples, num_slices_ecg, replacement_method)

In [ ]:
plot_lime(series_ecg, num_slices_ecg, X_test, y_test, BrS_predictions, idx_ecg, exp)

In [ ]:
stop here

# Label reproducibility per patient

In [ ]:
stripped= []
for p in test:
     stripped.append(p.split("_", 1)[0]) #remove everythin after "_"

stripped = list(dict.fromkeys(stripped))

In [ ]:
test_df = pd.DataFrame(test)
test_df.columns = ["ecg_id"]
test_df = pd.Series(test_df.ecg_id) 
dim = (2500, 8)
mini_x = np.empty((1, dim[0], dim[1]))

file_id_conf_mat = {"TN":[], "TP":[], "FN": [], "FP":[]}
p_id_reprod = {}

for p in stripped:
    all_samples =  list(test_df.loc[test_df.str.contains(p)].values)
    TP=0
    TN=0
    FP=0
    FN=0
    
    for s in all_samples:
        
        mini_x[0,] = np.load(samples_path + s +".npy")
        mini_y = labels[s] 
        if mini_y == 0: 
            mini_y = [1,0]
        if mini_y == 1:
            mini_y = [0,1]
   
        #predict and get performance metrics    
        mini_pred_probas = model.predict(mini_x)

        #BrS appears as 1, hence transformed to [0,1] => the second column returns 1 if BrS, 0 otherwise
        mini_BrS = mini_y[1]

        #get probabilities and predictions
        mini_BrS_probas = mini_pred_probas[:,1]
        mini_BrS_predictions = mini_pred_probas.argmax(axis = -1)
        
        
        if mini_BrS == 0:
            if mini_BrS_predictions == 0:
                TN = TN +1
                file_id_conf_mat["TN"].append(s)
                
                
            if mini_BrS_predictions == 1:
                FP = FP +1
                file_id_conf_mat["FP"].append(s)
        
        if mini_BrS == 1:
            if mini_BrS_predictions == 1:
                TP = TP +1
                file_id_conf_mat["TP"].append(s)
                
            if mini_BrS_predictions == 0:
                FN = FN +1
                file_id_conf_mat["FN"].append(s)
                
    p_id_reprod[p] = [labels[s], TN, FP, TP, FN]

In [ ]:
data = p_id_reprod
reprod = pd.DataFrame.from_dict(data, orient='index',
                       columns=['label', 'TN', 'FP', 'TP', 'FN'])

In [ ]:
reprod["number_of_samples"] = reprod["TN"] + reprod["TP"] + reprod["FN"] + reprod["FP"]
reprod["fraction_correct_labels"] = (reprod["TN"] + reprod["TP"]) / reprod["number_of_samples"]
reprod["all_samples_correctly_predicted"] = np.where(reprod["fraction_correct_labels"]== 1, True, False)
reprod

In [ ]:
print("number of different negative test patients: ", reprod[reprod["label"]==0].shape[0], " number of negative samples: ",sum(reprod[reprod["label"]==0]["number_of_samples"])) 
print("number negative samples per patient: ", sum(reprod[reprod["label"]==0]["number_of_samples"])/reprod[reprod["label"]==0].shape[0])

In [ ]:
print("number of different positive test patients: ", reprod[reprod["label"]==1].shape[0], " number of positive samples: ",sum(reprod[reprod["label"]==1]["number_of_samples"]))
print("number positive samples per patient: ", sum(reprod[reprod["label"]==1]["number_of_samples"])/reprod[reprod["label"]==1].shape[0])

In [ ]:
print("number of samples for positive patients")
sum(reprod[reprod["label"]==1]["number_of_samples"])

In [ ]:
print("number of samples for negative patients")
sum(reprod[reprod["label"]==0]["number_of_samples"])

## Drop patients with less than two samples

In [ ]:
reprod = reprod[reprod["number_of_samples"]>=2]

In [ ]:
print("number of different negative test patients: ", reprod[reprod["label"]==0].shape[0], " number of negative samples: ",sum(reprod[reprod["label"]==0]["number_of_samples"])) 
print("number negative samples per patient: ", sum(reprod[reprod["label"]==0]["number_of_samples"])/reprod[reprod["label"]==0].shape[0])

In [ ]:
print("number of different positive test patients: ", reprod[reprod["label"]==1].shape[0], " number of positive samples: ",sum(reprod[reprod["label"]==1]["number_of_samples"]))
print("number positive samples per patient: ", sum(reprod[reprod["label"]==1]["number_of_samples"])/reprod[reprod["label"]==1].shape[0])

In [ ]:
print("number of samples for positive patients")
sum(reprod[reprod["label"]==1]["number_of_samples"])

In [ ]:
print("number of samples for negative patients")
sum(reprod[reprod["label"]==0]["number_of_samples"])

## Fraction of correct labels : within patient agreement

In [ ]:
print("General fraction of correct labels")
print(np.mean(reprod["fraction_correct_labels"]), np.median(reprod["fraction_correct_labels"]))

In [ ]:
fig = plt.figure(figsize =(10, 10))
ax = fig.add_axes([0, 0, 1, 1]) 
bp = ax.boxplot(reprod["fraction_correct_labels"]) 
ax.set_xticklabels(['All groups'])
plt.title("Distribution of fraction of correct labels")
plt.show()

In [ ]:
data_1 = reprod[reprod["label"]==0]["fraction_correct_labels"]
data_2 = reprod[reprod["label"]==1]["fraction_correct_labels"]
df = [data_1, data_2]
fig = plt.figure(figsize =(10, 10)) 
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['No BrP', 'BrP'])
bp = ax.boxplot(df)
plt.title("Distribution of fraction of correct labels for positive and negative samples")
plt.show()

In [ ]:
print( "mean ratio of correct predictions per patient, positives: ", np.mean(reprod[reprod["label"]==1]["fraction_correct_labels"]),
     ", negatives: ", np.mean(reprod[reprod["label"]==0]["fraction_correct_labels"]))

## 100%-within patient agreement

In [ ]:
print("number of samples per patient for patients for which at least one prediction was wrong")
print(np.mean(reprod[reprod["all_samples_correctly_predicted"]==False]["number_of_samples"]), 
      np.median(reprod[reprod["all_samples_correctly_predicted"]==False]["number_of_samples"]))

In [ ]:
print("number of samples per patient for patients for which all predictions were right")
print(np.mean(reprod[reprod["all_samples_correctly_predicted"]==True]["number_of_samples"]), 
      np.median(reprod[reprod["all_samples_correctly_predicted"]==True]["number_of_samples"]))

In [ ]:
#boxplots
data_1 = reprod[reprod["all_samples_correctly_predicted"]==False]["number_of_samples"]
data_2 = reprod[reprod["all_samples_correctly_predicted"]==True]["number_of_samples"]
df = [data_1, data_2]
fig = plt.figure(figsize =(10, 7)) 
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['>1 incorrect prediction', 'All correct'])
bp = ax.boxplot(df)
plt.title("Distribution of number of samples with respect to whether all samples were correctly classified for each patient")
plt.show()

In [ ]:
correct = reprod[reprod["all_samples_correctly_predicted"]==True] #patients for which 100% within patient agreement was obtained
incorrect = reprod[reprod["all_samples_correctly_predicted"]==False] #patients for which less than 100% within patient agreement was obtained

In [ ]:
print("propotion of samples that reached 100% within patient agreement ", correct.shape[0]/(correct.shape[0]+incorrect.shape[0]))

In [ ]:
correct

In [ ]:
print("number of positive samples per patient for whom all ecgs were correctly classified")
print(np.mean(correct[correct["label"]==1]["number_of_samples"]), 
      np.median(correct[correct["label"]==1]["number_of_samples"]))

print("fraction of positive samples for which all sample predictions agreed ",correct[correct["label"]==1].shape[0] / (correct[correct["label"]==1].shape[0] + incorrect[incorrect["label"]==1].shape[0]))

In [ ]:
print("number of negative samples per patient for whom all ecgs were correctly classified")
print(np.mean(correct[correct["label"]==0]["number_of_samples"]), 
      np.median(correct[correct["label"]==0]["number_of_samples"]))
print("fraction of negative samples for which all sample predictions agreed ",correct[correct["label"]==0].shape[0] / (correct[correct["label"]==0].shape[0] + incorrect[incorrect["label"]==0].shape[0]))

In [ ]:
print("number of negative samples per patient for whom all at least one ecg was incorrectly classified")
print(np.mean(incorrect[incorrect["label"]==0]["number_of_samples"]), 
      np.median(incorrect[incorrect["label"]==0]["number_of_samples"]))

In [ ]:
print("number of positive samples per patient for whom all at least one ecg was incorrectly classified")
print(np.mean(incorrect[incorrect["label"]==1]["number_of_samples"]), 
      np.median(incorrect[incorrect["label"]==1]["number_of_samples"]))

In [ ]:
#boxplots
data_1 = correct[correct["label"]==1]["number_of_samples"]
data_2 = correct[correct["label"]==0]["number_of_samples"]
data_3 = incorrect[incorrect["label"]==1]["number_of_samples"]
data_4 = incorrect[incorrect["label"]==0]["number_of_samples"]

df = [data_1, data_2, data_3, data_4]
fig = plt.figure(figsize=(10,8)) 
ax = fig.add_axes([0.1, 0.1, 0.75, 0.75])
ax.set_xticklabels(["All correct and BrP", "All correct and no BrP", ">1 incorrect and BrP", ">1 incorrect and no BrP"])
ax.set_ylabel("Number of samples per patient ")
ax.boxplot(df)
#plt.title("Distribution of number of samples with respect to whether all samples were correctly classified for one patient per true label")
fig.savefig(plot_path + "BoxPlot_all_correct_at_least_one_wrong_per_class.png")
fig.show()

## New AUC

In [ ]:
fig, ax= plt.subplots(figsize = (10,10))
colors = {0:"green", 1: "red"}
labels = {0: "no BrP", 1: "BrP"}
grouped = reprod.groupby("label")
for key, group in grouped:
    group.plot(ax = ax, kind ="scatter", x = "number_of_samples", y= "fraction_correct_labels", label = labels[key], color = colors[key], s=50)
ax.set(xlabel = "Samples available per patient", ylabel = "Fraction of correctly predicted labels")
plt.rc("axes", labelsize=25)
plt.rc("legend", fontsize=20)
plt.rc("xtick", labelsize = 20)
plt.rc("ytick", labelsize = 20)
plt.savefig(plot_path + "scatter_fraction_correct_per_n_samples.png")

In [ ]:
gp_avg_fraction_correct_labels = reprod.groupby("number_of_samples", as_index=False)["fraction_correct_labels"].mean()
plt.scatter(gp_avg_fraction_correct_labels["number_of_samples"], gp_avg_fraction_correct_labels["fraction_correct_labels"])
plt.title("Scatter plot of average fraction of correctly predicted labels grouped by the number of samples available per patient")
plt.xlabel("Samples available per patient")
plt.ylabel("Average fraction of correctly predicted labels")

In [ ]:
a = reprod[reprod["label"]==1]
plt.scatter(a["number_of_samples"], a["fraction_correct_labels"])
plt.title("Scatter plot of fraction of correctly predicted labels with respect to the number of samples available per patient")
plt.xlabel("Samples available per patient")
plt.ylabel("Fraction of correctly predicted labels")

In [ ]:
gp_avg_fraction_correct_labels = a.groupby("number_of_samples", as_index=False)["fraction_correct_labels"].mean()
plt.scatter(gp_avg_fraction_correct_labels["number_of_samples"], gp_avg_fraction_correct_labels["fraction_correct_labels"])
plt.title("Scatter plot of average fraction of correctly predicted labels grouped by the number of samples available per patient")
plt.xlabel("Samples available per patient")
plt.ylabel("Average fraction of correctly predicted labels")

In [ ]:
b = reprod[reprod["label"]==0]
plt.scatter(b["number_of_samples"], b["fraction_correct_labels"])
plt.title("Scatter plot of fraction of correctly predicted labels with respect to the number of samples available per patient")
plt.xlabel("Samples available per patient")
plt.ylabel("Fraction of correctly predicted labels")

In [ ]:
gp_avg_fraction_correct_labels = b.groupby("number_of_samples", as_index=False)["fraction_correct_labels"].mean()
plt.scatter(gp_avg_fraction_correct_labels["number_of_samples"], gp_avg_fraction_correct_labels["fraction_correct_labels"])
plt.title("Scatter plot of average fraction of correctly predicted labels grouped by the number of samples available per patient")
plt.xlabel("Samples available per patient")
plt.ylabel("Average fraction of correctly predicted labels")

In [ ]:
# make new probabilities
#if positive label probability of BrP is fraction of correct
#if negative label, probability of BrP is 1- fraction of correct
reprod["new_probas"] = ""
reprod.loc[reprod["label"]==1, "new_probas"] = reprod["fraction_correct_labels"]
reprod.loc[reprod["label"]==0, "new_probas"] = 1 - reprod["fraction_correct_labels"]
reprod

In [ ]:
# predict based on fraction of correctly predicted
original_labels = reprod["label"]
new_probas = reprod["new_probas"]
#new_predictions = reprod["new_probas"].astype("float").round(0)
new_predictions = [1 if elem >= 0.5 else 0 for elem in new_probas]
metrics =[]
performance_metrics(original_labels,new_predictions, new_probas, metrics)

In [ ]:
#ROC curve
fpr, tpr, thresholds = roc_curve(original_labels, new_probas)
auc_coef = round(auc(fpr, tpr),3)
f, ax = plt.subplots(figsize=(6,6))
ax.plot(fpr, tpr, marker=".", label = model_type["type"] + " - AUC: " + str(auc_coef))
ax.plot([0,1], [0,1], transform = ax.transAxes, linestyle="--", label="Random Classifier")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC")
ax.legend()
plt.savefig(plot_path + "ROC_PP.png")

In [ ]:
#Precision Recall curve
precision, recall, thresholds = precision_recall_curve(original_labels, new_probas)
auprc = round(auc(recall, precision),3)
f, ax = plt.subplots(figsize=(6,6))
ax.plot(recall, precision, marker=".", label = model_type["type"] + " - AUPRC: " + str(auprc))
ax.set_xlabel("Recall (Positive label: Brugada)")
ax.set_ylabel("Precision (Positive label: Brugada)")
ax.set_title("AUPRC")
ax.set_ylim([0.0, 1.05])
ax.legend()
plt.savefig(plot_path + "AUPC_PP.png")

In [ ]:
calibration_together(original_labels.astype(float), new_probas.astype(float), per_patient = True)

In [ ]:
distribution(original_labels, {"Predicted probabilities": np.array(new_probas)}, per_patient = True)

# Filters data analysis

In [ ]:
neg_filters["combination"] = list(zip(neg_filters.high_pass, neg_filters.low_pass, neg_filters.ac))
pos_filters["combination"] = list(zip(pos_filters.high_pass, pos_filters.low_pass, pos_filters.ac))

In [ ]:
neg_filters = neg_filters.set_index('id')
pos_filters = pos_filters.set_index('id')

In [ ]:
neg_filters["classification_result"]=""
pos_filters["classification_result"]=""

In [ ]:
for key in file_id_conf_mat:
    for ecg_id in file_id_conf_mat[key]:
                
        if ecg_id in neg_filters.index:
            neg_filters.loc[ecg_id, "classification_result"] = key
        
        if ecg_id in pos_filters.index:
            pos_filters.loc[ecg_id, "classification_result"] = key

In [ ]:
neg_filters

In [ ]:
neg_filters['binary_result'] = np.where(neg_filters["classification_result"]== "TN", 1, 0)
pos_filters['binary_result'] = np.where(pos_filters["classification_result"]== "TP", 1, 0)

In [ ]:
neg_n_per_filter = neg_filters.groupby(["combination"]).size().reset_index(name="n_per_filter")
pos_n_per_filter = pos_filters.groupby(["combination"]).size().reset_index(name="n_per_filter")

In [ ]:
neg_correct_per_filter = pd.DataFrame(neg_filters.groupby(['combination'])['binary_result'].sum())
pos_correct_per_filter = pd.DataFrame(pos_filters.groupby(['combination'])['binary_result'].sum())

In [ ]:
neg_correct_per_filter = neg_correct_per_filter.rename(columns={"binary_result": "n_correct"})
pos_correct_per_filter = pos_correct_per_filter.rename(columns={"binary_result": "n_correct"})

In [ ]:
neg_n_per_filter = neg_n_per_filter.set_index('combination').join(neg_correct_per_filter)
pos_n_per_filter = pos_n_per_filter.set_index('combination').join(pos_correct_per_filter)

In [ ]:
neg_n_per_filter["n_incorrect"] = neg_n_per_filter["n_per_filter"]  - neg_n_per_filter["n_correct"]
pos_n_per_filter["n_incorrect"] = pos_n_per_filter["n_per_filter"]  - pos_n_per_filter["n_correct"]

In [ ]:
neg_n_per_filter["correct_over_n"] = neg_n_per_filter["n_correct"] / neg_n_per_filter["n_per_filter"]
pos_n_per_filter["correct_over_n"] = pos_n_per_filter["n_correct"] / pos_n_per_filter["n_per_filter"]

In [ ]:
pos_n_per_filter = p_filter_combo[["combination", "percentage_by_class"]].set_index('combination').join(pos_n_per_filter)

In [ ]:
neg_n_per_filter = n_filter_combo[["combination", "percentage_by_class"]].set_index('combination').join(neg_n_per_filter)

In [ ]:
neg_n_per_filter.sort_values(by=['correct_over_n'], ascending = False)

In [ ]:
pos_n_per_filter.sort_values(by=['correct_over_n'], ascending = False)

In [ ]:
n_filter_combo = n_filter_combo.set_index("combination")
n_filter_combo = n_filter_combo.reindex(index = p_filter_combo["combination"])
n_filter_combo = n_filter_combo.reset_index()

In [ ]:
fig_index = 1
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 10))

ind = np.arange(pos_n_per_filter.shape[0])
width = 0.35

rects_neg = ax1.bar(ind - width/2, neg_n_per_filter["percentage_by_class"], width, label = "No BrP")
rects_pos = ax1.bar(ind + width/2, pos_n_per_filter["percentage_by_class"], width, label = "BrP")
ax1.set_ylabel("Percentage of samples")
ax1.set_xlabel("Filter combinations")
#ax1.set_title("Percentage of samples per filter combination per class")
ax1.set_xticks(ind)
y_labels = list(neg_n_per_filter.index)
ax1.set_xticklabels(y_labels)
ax1.legend()

rects_neg = ax2.bar(ind - width/2, neg_n_per_filter["correct_over_n"], width, label = "No BrP")
rects_pos = ax2.bar(ind + width/2, pos_n_per_filter["correct_over_n"], width, label = "BrP")
ax2.set_ylabel("Ratio of correctly predicted samples per filter combination")
#ax2.set_title("Ratio of correctly predicted samples per filter combination per class")
ax2.set_xticks(ind)
y_labels = list(neg_n_per_filter.index)
ax2.set_xticklabels(y_labels)
ax2.set_xlabel("Filter combinations")
ax2.legend()

plt.savefig(plot_path + "performance_per_filter.png")

## Performance metrics per filter

In [ ]:
data = neg_filters[["combination", "classification_result"]].append(pos_filters[["combination", "classification_result"]])


In [ ]:
data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data["classification_result"])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
data[['FN', 'FP', 'TN', 'TP']]=onehot_encoded

In [ ]:
data

In [ ]:
filter_performance = pd.DataFrame()

In [ ]:
filter_performance["FN"] = data.groupby('combination')['FN'].sum()
filter_performance["FP"] = data.groupby('combination')['FP'].sum()
filter_performance["TN"] = data.groupby('combination')['TN'].sum()
filter_performance["TP"] = data.groupby('combination')['TP'].sum()
filter_performance["PPV/precision"] = filter_performance["TP"]/(filter_performance["TP"]+filter_performance["FP"])
filter_performance["TNR/specificity"] = filter_performance["TN"]/(filter_performance["TN"]+filter_performance["FP"])
filter_performance["FNR/specificity"] = filter_performance["FN"]/(filter_performance["FN"]+filter_performance["TP"])

In [ ]:
filter_performance

In [ ]:
#export predictions to csv
labels_and_predictions_p_sample = pd.DataFrame(list(zip(BrS, BrS_probas)), columns = ["label_per_sample", "prediction_per_sample"])
labels_and_predictions_p_patient = pd.DataFrame(list(zip(original_labels, new_probas)), columns = ["label_per_sample", "prediction_per_sample"])
labels_and_predictions_p_sample.to_csv("ecg_predictions_per_sample.csv", index=False)
labels_and_predictions_p_patient.to_csv("ecg_predictions_per_patient.csv", index=False)